In [1]:
import numpy
import math 
import matplotlib.pyplot as plt
import networkx as nx

In [2]:
def genRandInteger(a, b, cant = None):
    ''' This function generates a random integer in the integer range [a, b]'''
    import numpy, math
    
    if cant != None:
        U = numpy.random.uniform(a,b)
        return math.floor(U)
    else:
        while(True):
            U = math.floor( numpy.random.uniform(a,b) )
            if U != cant:
                break
        return U
def detOpinionSize(attr):
    ''' Given alist of attributes in which each 
        element is list of size two where its second element is its 
        opinion and the first element is its node index, we output the number 
        of nodes of opinion 0 and nodes of opinion 1'''
    num_0 = num_1 = 0
    for i in range(len(attr)):
        if attr[i][1]['opinion'] == 0:
            num_0 += 1
        if attr[i][1]['opinion'] == 1:
            num_1 += 1
    return (num_0, num_1)
        

In [3]:
# Generate a Complete Graph!
numAgents = 20
numEdges = .5 * numAgents*(numAgents-1)
G = nx.gnm_random_graph(numAgents, numEdges)

#G = nx.complete_graph(numAgents)



In [ ]:
# initializaiton of attributes i.e initiatilization of binary opinion space
for i in range(numAgents):
    U = numpy.random.uniform(0,1)
    if U < .5:
        G.node[i] = {'opinion': 0}
    else:
        G.node[i] = {'opinion': 1}

attr = G.nodes(data = True)
attributes = nx.get_node_attributes(G, 'opinion')
            
from itertools import count
attributes = nx.get_node_attributes(G, 'opinion')
groups = set(nx.get_node_attributes(G,'opinion').values())
mapping = dict(zip(sorted(groups),count()))
nodes = G.nodes()
colors = [mapping[G.node[n]['opinion']] for n in nodes]

# drawing nodes and edges separately so we can capture collection for colobar
pos = nx.spring_layout(G)
ec = nx.draw_networkx_edges(G, pos, alpha=0.2)
nc = nx.draw_networkx_nodes(G, pos, nodelist=nodes, node_color=colors, 
                            with_labels=False, node_size=100, cmap=plt.cm.jet)
plt.colorbar(nc)
plt.axis('off')
plt.savefig('initial.png')
plt.show()

In [ ]:
# discretization of time 
t = numpy.linspace(0, 100, 1000)
dt = t[1] - t[0]

#begin discrete time process
p = .1

for (ithTime, time) in enumerate(t):
    
    U = numpy.random.uniform(0,1)
    num_0, num_1 = detOpinionSize(attr)
    # Part 1: Captures Media Influence; prob of p
    if U < p:
        u1 = numpy.random.uniform(0,1)
        
        # random number of 0 opinions update their opinions to 1 
        if u1 < .5:
            x = genRandInteger(0, num_0)
            for i in range(int(x)):
                G.node[i]['opinion'] = 1
            
        # random number of 1 opinion nodes update their opinions to 0
        else:
            x = genRandInteger(0, num_1)
            for i in range(int(x)):
                G.node[i]['opinion'] = 0
    
    # Part 2: captures the node-to-node interactions; prob of 1-p
    else:
        # chose a random agent
        agent_i = genRandInteger(0, numAgents)
    
        # chose a random agent that agent before chooses
        agent_j = genRandInteger(0, numAgents, cant = agent_i)
        
        # ith agent assumes the opinion of jth adgent if they're different
        if G.node[agent_i]['opinion'] != G.node[agent_j]['opinion']:
            G.node[agent_i]['opinion'] = G.node[agent_j]['opinion']
    if (ithTime % 3000) == 0:
        
        attributes = nx.get_node_attributes(G, 'opinion')
        groups = set(nx.get_node_attributes(G,'opinion').values())
        mapping = dict(zip(sorted(groups),count()))
        nodes = G.nodes()
        colors = [mapping[G.node[n]['opinion']] for n in nodes]

        # drawing nodes and edges separately so we can capture collection for colobar
        pos = nx.spring_layout(G)
        ec = nx.draw_networkx_edges(G, pos, alpha=.1)
        nc = nx.draw_networkx_nodes(G, pos, nodelist=nodes, node_color=colors, 
                                    with_labels=True, node_size=100, cmap=plt.cm.jet)
        plt.colorbar(nc)
        plt.axis('on')

        #plt.show()